In [3]:
import torch
import sys
import os
import os 
# backbone = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512")
from transformers.models.segformer import TwoStreamSegformerEncoder, SegformerConfig, TwoStreamSegformerModel, TwoStreamSegformerForSemanticSegmentation
TwoStreamSegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512")

TwoStreamSegformerForSemanticSegmentation(SegformerConfig())(torch.zeros(5, 3, 512, 512), torch.zeros(5, 3, 512, 512), torch.zeros(5, 3, 512, 512)).logits.shape

torch.Size([15, 2, 128, 128])

In [4]:
TwoStreamSegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512")


Some weights of TwoStreamSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b1-finetuned-ade-512-512 and are newly initialized: ['segformer.encoder.BCAs.0.linear1.bias', 'segformer.encoder.BCAs.0.linear1.weight', 'segformer.encoder.BCAs.0.linear2.bias', 'segformer.encoder.BCAs.0.linear2.weight', 'segformer.encoder.BCAs.0.norm1.bias', 'segformer.encoder.BCAs.0.norm1.weight', 'segformer.encoder.BCAs.0.norm2.bias', 'segformer.encoder.BCAs.0.norm2.weight', 'segformer.encoder.BCAs.0.self_attn.in_proj_bias', 'segformer.encoder.BCAs.0.self_attn.in_proj_weight', 'segformer.encoder.BCAs.0.self_attn.out_proj.bias', 'segformer.encoder.BCAs.0.self_attn.out_proj.weight', 'segformer.encoder.BCAs.1.linear1.bias', 'segformer.encoder.BCAs.1.linear1.weight', 'segformer.encoder.BCAs.1.linear2.bias', 'segformer.encoder.BCAs.1.linear2.weight', 'segformer.encoder.BCAs.1.norm1.bias', 'segformer.encoder.BCAs.1.norm1.weight', 'segformer.encoder.BCAs.1.norm2.bias

TwoStreamSegformerForSemanticSegmentation(
  (segformer): TwoStreamSegformerModel(
    (encoder): TwoStreamSegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-0

In [12]:
import numpy as np 
x = np.array(
    [[[1, 2, 3],[4, 5, 6]],[[11, 2, 3],[4, 5, 6]], [[7, 8, 9],[10, 11, 12]]]
)
y = np.array(
    [[[-1, -2, -3],[-4, -5, -6]],[[-11, -2, -3],[-4, -5, -6]], [[-7, -8, -9],[-10, -11, -12]]]
)
z = np.concatenate([x, y], axis=0)
z.shape
z.reshape(2, 3, 2, 3)[0]

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[11,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]])

In [ ]:
# backbone.segformer.encoder.patch_embeddings[0].proj = torch.nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
backbone.segformer.encoder.block[0]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
backbone2 = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512")
class BCA(nn.Module):
    """
    Background-CurrentFrame Attention
    """
    def __init__(self, dim=384):
        super(BCA, self).__init__()
        self.dim = dim
        self.key_projection = nn.Linear(dim, dim)
        self.query_projection = nn.Linear(dim, dim)
        self.value_projection = nn.Linear(dim * 2, dim)
        self.projection_dropout = nn.Dropout(0.2)
        self.mlp = nn.Sequential(
            nn.Linear(2 * dim, 1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.Dropout(0.2), 
            nn.ReLU(), 
            nn.Linear(1024, dim),
            nn.Dropout(0.2)
        )
        self.norm1 = nn.LayerNorm(dim) 
        self.norm2 = nn.LayerNorm(dim)
        self.head = nn.Linear(dim * 2, dim)

    def forward(self, current_frame, long_background, short_background):
        """
        long_background: torch.Tensor, shape (batch, L, dim)
        short_background: torch.Tensor, shape (batch, L, dim)
        current_frame: torch.Tensor, shape (batch, L, dim)
        """
        key = self.projection_dropout(self.key_projection(long_background))
        query = self.projection_dropout(self.query_projection(current_frame))
        value = self.projection_dropout(self.value_projection(torch.concatenate([long_background, current_frame], dim=-1)))

        attn_score = torch.einsum("bld,bld->bl", query, key) / self.dim**0.5
        attn_score = F.softmax(attn_score, dim=1)
        attn_output = torch.einsum("bl,bld->bld", attn_score, value)
        attn_output = self.norm1(attn_output + current_frame) 
        mlp_output = self.mlp(torch.concatenate([attn_output, current_frame], dim=-1))
        mlp_output = self.norm2(mlp_output + attn_output)

        # short background attention with frame
        key = self.projection_dropout(self.key_projection(short_background))
        query = self.projection_dropout(self.query_projection(current_frame))
        value = self.projection_dropout(self.value_projection(torch.concatenate([short_background, current_frame], dim=-1)))

        attn_score = torch.einsum("bld,bld->bl", query, key) / self.dim**0.5
        attn_score = F.softmax(attn_score, dim=1)
        attn_output = torch.einsum("bl,bld->bld", attn_score, value) 
        attn_output = self.norm1(attn_output + mlp_output)
        mlp_output = self.mlp(torch.concatenate([attn_output, mlp_output], dim=-1))
        mlp_output2 = self.norm2(mlp_output + attn_output)
        return self.head(torch.cat([mlp_output, mlp_output2], dim=-1))
        
class NewSegformerLayer(torch.nn.Module):
    def __init__(self, original_layer):
        super().__init__()
        self.original_layer = original_layer
        self.bca = BCA()

    def forward(self, hidden_states, height, width, output_attention):
        x = self.original_layer(hidden_states, height, width, output_attention)
        print(len(x))
        return self.new(x)
    
backbone2.segformer.encoder.block[0][0] = NewSegformerLayer(backbone.segformer.encoder.block[0][0])
rand_x = torch.randn(64, 3, 256, 256)
assert not torch.allclose(backbone(rand_x).logits, backbone2(rand_x).logits)
